In [68]:
### training data Preprocess
import pandas as pd
import datetime as date
import numpy as np
TrainData = pd.read_csv('D:/.BDT/5001/msbd5001-fall2020/train.csv', parse_dates = ['date'], dayfirst= True)
print(TrainData.describe())
print(TrainData.head())

                id         speed
count  14006.00000  14006.000000
mean    7002.50000     32.779118
std     4043.32827     13.573813
min        0.00000      2.573417
25%     3501.25000     19.301089
50%     7002.50000     36.580595
75%    10503.75000     45.877665
max    14005.00000     53.161286
   id                date      speed
0   0 2017-01-01 00:00:00  43.002930
1   1 2017-01-01 01:00:00  46.118696
2   2 2017-01-01 02:00:00  44.294158
3   3 2017-01-01 03:00:00  41.067468
4   4 2017-01-01 04:00:00  46.448653


In [69]:
# 公共假期
vacation = ['20170101','20170128','20170130','20170131','20170404','20170501','20170530','20170701','20171001','20171005',
            '20171028','20171222','20171225','20180101','20180216','20180217','20180219','20180405','20180501','20180618',
            '20180701','20180925','20181001','20181017','20181222','20181225' ]
# 日期提取
def extract_date(df,col):
    df["year"] = df[col].apply(lambda x: x.year)
    df["month"] = df[col].apply(lambda x: x.month)
    df["day"] = df[col].apply(lambda x: x.day)
    df["hour"] = df[col].apply(lambda x: x.hour)
    df["is_sunday"] = df[col].apply(lambda x : 1 if x.weekday() in [6] else 0)
    df["dayofweek"] = df[col].apply(lambda x : x.weekday()+1)
    df["tmp"] = df[col].apply(lambda x: x.strftime('%Y%m%d'))
    df["is_holiday"] = df["tmp"].apply(lambda x: 1 if x in vacation else 0)
    
extract_date(TrainData,'date')
TrainData.head(160)


,id,date,speed,year,month,day,hour,is_sunday,dayofweek,tmp,is_holiday
0,0,2017-01-01 00:00:00,43.002930,2017,1,1,0,1,7,20170101,1
1,1,2017-01-01 01:00:00,46.118696,2017,1,1,1,1,7,20170101,1
2,2,2017-01-01 02:00:00,44.294158,2017,1,1,2,1,7,20170101,1
3,3,2017-01-01 03:00:00,41.067468,2017,1,1,3,1,7,20170101,1
4,4,2017-01-01 04:00:00,46.448653,2017,1,1,4,1,7,20170101,1
...,...,...,...,...,...,...,...,...,...,...,...
155,155,2017-01-07 11:00:00,13.308467,2017,1,7,11,0,6,20170107,0
156,156,2017-01-07 12:00:00,19.686580,2017,1,7,12,0,6,20170107,0
157,157,2017-01-07 13:00:00,17.374724,2017,1,7,13,0,6,20170107,0
158,158,2017-01-07 14:00:00,26.702782,2017,1,7,14,0,6,20170107,0


In [70]:
TrainData.drop(columns=["id", "date","tmp"], inplace=True)
TrainData.head()

,speed,year,month,day,hour,is_sunday,dayofweek,is_holiday
0,43.002930,2017,1,1,0,1,7,1
1,46.118696,2017,1,1,1,1,7,1
2,44.294158,2017,1,1,2,1,7,1
3,41.067468,2017,1,1,3,1,7,1
4,46.448653,2017,1,1,4,1,7,1


In [71]:
#  test data Preprocess
TestData = pd.read_csv('D:/.BDT/5001/msbd5001-fall2020/test.csv', parse_dates = ['date'], dayfirst= True)
print(TestData.describe())
extract_date(TestData,'date')
TestData.drop(columns=["id", "date","tmp"], inplace=True)
TestData.head()

                id
count  3504.000000
mean   1751.500000
std    1011.661999
min       0.000000
25%     875.750000
50%    1751.500000
75%    2627.250000
max    3503.000000


,year,month,day,hour,is_sunday,dayofweek,is_holiday
0,2018,1,1,2,0,1,1
1,2018,1,1,5,0,1,1
2,2018,1,1,7,0,1,1
3,2018,1,1,8,0,1,1
4,2018,1,1,10,0,1,1


In [72]:
# Models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression,SGDRegressor
from sklearn.linear_model import Ridge, RidgeCV, Lasso
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.svm import SVR
from mlxtend.regressor import StackingCVRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

In [73]:
X_train = TrainData.drop(columns=["speed"])
Y_train = TrainData["speed"]
X_test = TestData

In [74]:
# XGBRegressor
other_params = {'learning_rate': 0.1, 'n_estimators': 50, 'max_depth': 4, 'min_child_weight': 1, 'seed': 0,
                    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0.5, 'reg_alpha': 3, 'reg_lambda': 3}
# xgb = XGBRegressor(**other_params)
xgb = XGBRegressor(n_estimators=40,n_jobs=-1)
xgb.fit(X_train, Y_train)
y_pred_train = xgb.predict(X_train)

# 评估回归性能
print('Mean Squared Error:', metrics.mean_squared_error(Y_train, y_pred_train))

Mean Squared Error: 9.896843299768069


In [75]:
from pandas.core.frame import DataFrame

y_pred_test = xgb.predict(X_test)
submit = {"speed": y_pred_test}
submit_data = DataFrame(submit)
# submit_data["speed"].to_list()
submit_data.to_csv("hjq5.csv", index_label="id")